In [19]:
import geemap, ee

In [20]:
#ee.Authenticate()
# Initialize the library.
ee.Initialize()

In [21]:
%load_ext autoreload
%autoreload 2
import sys  
sys.path.insert(0, '../lib/')



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
%reload_ext autoreload
import quality_mosaic as qm

# Choose date and location

## Define parameters

In [23]:
date = '2020-06-18'
lat, lon = 17.973859475,-65.988746993


In [24]:
lat, lon = 48.1496822,-123.5552726
date = '2020-10-15'

In [25]:
lat, lon = lat, lon = 33.869066751,-118.405718865
date = '2021-01-18'

## Define Hyperparameters

In [26]:
#define the total number of days before and after the date of interest you want to search``
max_search_window_months = 6

Convert to ee objects

In [27]:
point = ee.Geometry.Point([lon, lat])
sample = point.buffer(10).bounds()
roi = point.buffer(100).bounds()

In [28]:
DOI = ee.Date(date)

start_date, end_date = DOI.advance(-max_search_window_months, 'month'), DOI.advance(max_search_window_months, 'month')
start_date_str, end_date_str = start_date.format('YYYY-MM-dd').getInfo(), end_date.format('YYYY-MM-dd').getInfo()
print("Search window from {:} to {:}".format(start_date_str, end_date_str))


Search window from 2020-07-18 to 2021-07-18


In [29]:
s2_cld_col = qm.get_s2_sr_cld_col(roi, start_date_str, end_date_str)
visParamsRGB = {"min": 0, "max": 4000, "bands": ["B4", "B3", "B2"]}

witFScore = qm.wrap_addFScore(s2_cld_col, DOI)
qm_s2 = witFScore.qualityMosaic('FScore')
qm_s2.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B11',
 'B12',
 'AOT',
 'WVP',
 'SCL',
 'TCI_R',
 'TCI_G',
 'TCI_B',
 'MSK_CLDPRB',
 'MSK_SNWPRB',
 'QA10',
 'QA20',
 'QA60',
 'prob',
 'clouds',
 'delta',
 'FScore']

# Show interactive map

Add a marker manually if you want to change the location of the sample pointm

In [32]:
Map = geemap.Map()
Map.add_basemap('SATELLITE')
Map.addLayer(qm_s2, visParamsRGB, 'qm_s2')

Map.addLayer(sample,
             {'color':'yellow', 'opacity':0.3},
             name='buffer')

Map.add_marker([lat, lon],tooltip='sample', name='sample', draggable=False)
Map.centerObject(sample, 20)

Map

Map(center=[33.869066767472795, -118.40571869467622], controls=(WidgetControl(options=['position', 'transparen…

In [33]:
if (Map.draw_last_feature is not None):
    
    Map.draw_last_feature.geometry().getInfo()['coordinates']
    new_sample = Map.draw_last_feature.buffer(10).bounds()
    Map.addLayer(new_sample,
             {'color':'green', 'opacity':0.3},
             name='new_buffer')
# Map.draw_last_feature#.geometry().getInfo()['coordinates']

## TODOs:
1. Add S-1 imagery
2. Remove all unnecesary bands
3. Add mTGSI and BSI indices
4. Try the reduceRegion to capture info within sample buffer